<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_answer_KCBERT_nsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질문에 대한 답변을 선별하는 코드(BERT Next Sentence Prediction)(로컬)

In [ ]:
!pip install soynlp
!pip install datasets
!pip install accelerate -U
import accelerate
import random
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from datasets import Dataset, load_dataset, ClassLabel
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM, Trainer, TrainerCallback
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#NSP로 훈련된 모델 로드
finetuned_model_load_path = '/content/drive/My Drive/Finetuned_Model_judge_answer'

finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_load_path)
finetuned_model = BertForNextSentencePrediction.from_pretrained(finetuned_model_load_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#카카오톡 데이터 불러오기
file_path = '/content/drive/My Drive/judge_question_result_short_KcBERT.csv'

In [ ]:
#카카오톡 대화내용을 데이터프레임으로 받기
df = pd.read_csv(file_path)

#질문으로 판별된 텍스트를 새 데이터프레임으로 생성
df_question = df[df['label'] == 'question']

print(df_question)
#새 데이터프레임의 text, index, name, date를 딕셔너리에 저장
# questions_index_dict = {text : index for (text, index) in zip(df_question['text'], df_question.index)}
# print(questions)

question_index_creator_date_dict = {text : {'index' : index, 'name' : name, 'date' : date} for (text, index, name, date) in zip(df_question['text'], df_question.index, df_question['name'], df_question['date'])}
print(question_index_creator_date_dict)

      Unnamed: 0.1  Unnamed: 0                   name  \
0                0           0          조용찬 / 백엔드 개발자   
3                3           3               조한울 / 운영   
10              10          10               IT / 돔돔철   
29              29          30               IT / 돔돔철   
34              34          35               김호종/스타트업   
...            ...         ...                    ...   
7845          7845        7870                 퐈이팅해야지   
7848          7848        7873               버블챌린지하실분   
7849          7849        7874          하헌준 / 백엔드 개발자   
7850          7850        7875   안상영/NOSEO Technology   
7852          7852        7877                    김승진   

                                                   text        date     label  
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...  2023-06-15  question  
3      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...  2023-06-15  question  
10     혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;  2023-06-15  questi

In [ ]:
# Ensure using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = finetuned_model.to(device)
tokenizer = finetuned_tokenizer

cuda


In [ ]:
#중복 질문 제거를 동시에 수행하기 위한 새로운 알고리즘

#실험을 위한 하이퍼파라미터 설정
#후보군으로 삼을 텍스트 개수 범위
text_range = 20
#답변 목록에 추가할지 기준이 되는 레이블값
standard = 0.5

#질문-답변 쌍 딕셔너리 생성. index로 질문 번호를 관리하기 때문에 value를 리스트로 작성하고, 리스트 안에는 {'question' : 질문내용, 'answer' : 답변내용, 'questioner' : 작성자명, 'respondent' : 답변자 목록, 'date' : 작성날짜} 가 포함되도록 한다.
qa_pair_dictionary = {info['index'] : {'question' : question, 'answer' : [], 'questioner' : info['name'], 'respondent' : [], 'date' : info['date']} for (question, info) in question_index_creator_date_dict.items()}

#질문 속 질문인지 판별할 때 사용할 불리언
in_question_texts = False

all_texts = {index : {'text' : text, 'name' : name} for index, text, name in zip(df.index, df['text'], df['name'])}

for index, item in tqdm(all_texts.items(), desc = 'Processing Answer to Question'):

  candidate_qa_list = [] #현재 텍스트의 소속을 판정할 (질문-답변 딕셔너리) 리스트
  candidate_qa_index_list = []
  in_question_texts = True if df.iloc[index]['label'] == 'question' else False #판별할 텍스트가 질문인지 검사

  start = 0 if index < 20 else index - text_range - 1 #인덱스가 20 미만일 경우 검사 범위 조정
  for i in range(start, index): #현재 텍스트가 소속될 질문의 범위
    candidate = qa_pair_dictionary.get(i, None) #qa_pair_dictionary에서 i 인덱스에 해당하는 질답 딕셔너리 가져오기
    if candidate != None:
      candidate_qa_list.append(candidate) #결과 리스트에서 최대 확률인 질문을 인덱싱하기 위해 인덱스를 포함한 딕셔너리를 append
      candidate_qa_index_list.append(i)

  if len(candidate_qa_list) == 0:
    continue

  #데이터를 튜플로 묶은 뒤 배치처리
  batched_data = [(qa_dict['question'] + ' '.join(qa_dict['answer']), item['text']) for qa_dict in candidate_qa_list]
  inputs = tokenizer(batched_data, padding = True, truncation = True, return_tensors = 'pt')
  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

  outputs = [{'scores' : value, 'index' : index} for value, index in zip(probabilities.tolist(), candidate_qa_index_list)]

  # 'scores'의 두 번째 값(연속적인 문장일 확률)에 따라 내림차순으로 정렬
  sorted_output = sorted(outputs, key = lambda x : x['scores'][1], reverse=True)

  output = sorted_output[0] #텍스트가 소속될 질문

  if output['scores'][1] > standard:
    #모순이 발생하지 않기 위해서 질문 속 질문으로 판별된 경우 즉시 qa_pair_dictionary에서 해당 질문을 삭제해야 함.
    if in_question_texts == True:
        del qa_pair_dictionary[index]

    qa_pair_dictionary[output['index']]['answer'].append(item['text'])
    qa_pair_dictionary[output['index']]['respondent'].append(item['name'])

  # print('\n',qa_pair_dictionary)
  # print('\n',used_question_index)


Processing Answer to Question: 100%|██████████| 7859/7859 [05:27<00:00, 23.98it/s]


In [ ]:
print(qa_pair_dictionary)

{0: {'question': ' deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaPercent가 양수, 음수 모두 존재할 때 절대값으로 바꾸는 방법 아시는 분 있으실까요? 예를 들어 deltaPercent 값이 -1, -3, 2, 5 이렇게 존재할 때, 이를 모두 절대값으로 바꾼 후 5, 3, 2, 1 순서로 정렬하려고 합니다!', 'answer': [], 'questioner': ' 조용찬 / 백엔드 개발자', 'respondent': [], 'date': '2023-06-15'}, 3: {'question': ' 혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데, 곧 요금제를 업그레이드 안 하면 감당이 안 될 거 같은데요.계산해보니 예상 요금이 월 400만원이 나오게 생겼네요..ㅠㅠ', 'answer': [], 'questioner': ' 조한울 / 운영', 'respondent': [], 'date': '2023-06-15'}, 10: {'question': ' 혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;', 'answer': [' 저희는 히로인스라는 서비스를 운영하고 있는데요. 백엔드만 버블을 쓰고 있습니다.', ' 버블 프라이싱에 나와있는 워크로드 보면 초기 이후에는 쓰지 말라는 느낌 같네요..ㅎㅎ', ' Do a search for는 어지간하면 안 쓰는 방향으로..'], 'questioner': ' IT / 돔돔철', 'respondent': [' 조한울 / 운영', ' 조한울 / 운영', ' 배문형 / 프리랜스'], 'date': '2023-06-15'}, 29: {'question': ' 혹시 버블로 개발 완료해서 실서비스 운영중이신 분들 중에는 비용이 어떻게 나오는지 궁금합니다. 물론 case 별로 모두 다들 수는 있지만 그래도 참고는 될 것 같습니다.', 'answer': [], 'questioner': ' IT / 돔돔철', 'resp

In [ ]:
data_result= [item for item in qa_pair_dictionary.values()]

#결과 데이터를 데이터프레임으로 변환
df_result = pd.DataFrame(data = data_result, columns = ['question', 'answer', 'questioner', 'respondent', 'date'])
print(df_result)

                                               question  \
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
2      혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
3      혹시 버블로 개발 완료해서 실서비스 운영중이신 분들 중에는 비용이 어떻게 나오는지...   
4                                 10번 할거 1번으로 어떻게 끝내나요?   
...                                                 ...   
1598      혹시 버블 증상이 로딩이 오래 걸리는거 말씀하시나용 저도 지금 버블이 안켜져용ㅋㅋ   
1599   혹시 포스기 소리를 제작 혹은 외주를 맡겨보신 경험이 있는 분 이곳에 계실까요? ...   
1600                           안녕하세요 혹시 버블 공부하시는분 계신가요?   
1601   혹시 버블에서 도움말 기능을 만들고 싶은데, 추천하시는 강의가 있나요??도움말 기...   
1602   Workflow error - List at maximum length (1000...   

                                                 answer      questioner  \
0                                                    []   조용찬 / 백엔드 개발자   
1                                                    []        조한울 / 운영   
2     [ 저희는 히로인스라는 서비스를 운영하고 있는데요. 백엔드만 버블을 쓰고 있습니다....        IT 

In [ ]:
#answer가 비어있는 행은 제거
df_result = df_result[df_result['answer'].apply(lambda x: x != [])]
print(len(df_result))

577


In [ ]:
print(df_result.iloc[0]['answer'] == [])

True


In [ ]:
print(df_result)

                                               question  \
2      혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
8      API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버블 ...   
9      안녕하세요~ 프로모션 적용하려면 플랜 업그레이드를 별도로 진행해야하는걸까요 아니면...   
13     감사합니다. 버블에서도 매 n시간 단위로 특정 api 콜을 예약하는 loop를 만...   
15     혹시 워드프레스도 노코드 툴 중 하나인가요? 버블 softr 웹플로우 아임웹 글라...   
...                                                 ...   
1597   안녕하세요. 제작 중이던 서비스 이름이 변경되면서 앱 이름을 바꾸려고 하는데 아무...   
1598      혹시 버블 증상이 로딩이 오래 걸리는거 말씀하시나용 저도 지금 버블이 안켜져용ㅋㅋ   
1599   혹시 포스기 소리를 제작 혹은 외주를 맡겨보신 경험이 있는 분 이곳에 계실까요? ...   
1601   혹시 버블에서 도움말 기능을 만들고 싶은데, 추천하시는 강의가 있나요??도움말 기...   
1602   Workflow error - List at maximum length (1000...   

                                                 answer      questioner  \
2     [ 저희는 히로인스라는 서비스를 운영하고 있는데요. 백엔드만 버블을 쓰고 있습니다....        IT / 돔돔철   
8     [  API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버...    김범준 / 기획, 개발   
9     [ 안녕하세요? 프로모션은 자동 적용됩니다! 단, 콜수나 유저수에 따라 플랜 비용은...        기뻐하

In [ ]:
#csv 파일로 google drive에 저장
#long 파일의 경우 illegalcharactererror로 인해서 .csv 파일로 저장하기
save_path = '/content/drive/My Drive/judge_answer_result_KcBERT_short_questiontrained.csv'

df_result.to_csv(save_path)